####IMPORT SCRAP

In [ ]:
import numpy as np
import pandas as pd

import requests
import csv
from bs4 import BeautifulSoup as bs
import lxml

from urllib.request import urlopen

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#Create Chrome Options - Necessary for Colab
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',options=options)
from selenium.webdriver.common.keys import Keys

import unittest

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import re
from scrapy import Selector   

####IMPORT TWITTER

In [ ]:
import tweepy
from textblob import TextBlob
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords                            
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist

####IMPORT TELEGRAM

In [ ]:
import telebot
import datetime

####TOKENS

We add the tokens we will need for both Twitter and Telegram to some variables

In [ ]:
#TWITTER
consumerKey = YOUR TOKENS
consumerSecret = YOUR TOKENS
accessToken = YOUR TOKENS
accessTokenSecret = YOUR TOKENS

#TELEGRAM
id_channel = YOUR CHANNEL
telegram_token = YOUR TOKENS

##SCRAP

We proceed to scrape the investing page where we extract the name and volume values from the table.

The company name that gives us investing, is the one we will use later to search the Tweets in Telegram. 

In [ ]:
url = 'https://www.investing.com/equities/most-active-stocks?country=usa'
session = requests.Session()
request = session.get(url)
soup = bs(request.content,'lxml')

In [ ]:
nombre = []; volumen = [];

empresas = soup.find_all('td', class_='left bold plusIconTd elp', itemscope="")
volumen_empresas = soup.find_all('td', class_='turnover', itemscope="")

for empresa in empresas:
    nombre.append(empresa.text)

for EachPart in soup.select('td[class*="turnover"]'):
    volumen.append(EachPart.text)

We transform the extracted data into a dataframe to be able to work with the data in a simpler way.

In [ ]:
nombre = [x.lower() for x in nombre]

df_volumen = pd.DataFrame({
    'Nombre': nombre,
    'Volumen': volumen
})

##DATA FRAME

We transform the data in the volume column so that it is a number and not a string.

In [ ]:
df_volumen.dtypes
df_volumen['Volumen'] = df_volumen['Volumen'].str[:-1] #Eliminamos la M de la columna Volumen
df_volumen['Volumen'] = df_volumen['Volumen'].astype(float)

We download a table where each company name is related to its corresponding ticker, this will be useful when presenting the results in the Telegram message.

In [ ]:

url = 'https://raw.githubusercontent.com/Huanhotze/Stock_Twitter_Sentiment/main/stocks.xlsx'
stocks = pd.read_excel(url)
columns = ['Nombre', 'Ticker']
stocks.columns = columns

We will merge the 2 tables resulting in a table with the name, volume and ticker.

We will sort by value and finally we will be left with the 5 companies with the highest volume.

In [ ]:
df_tickers = pd.merge(df_volumen, 
                      stocks, 
                      on ='Nombre', 
                      how ='inner')

In [ ]:
df_tickers = df_tickers.sort_values(by=['Volumen'], ascending=False)

In [ ]:
first_n_column  = df_tickers.iloc[:5]
tickers = first_n_column.Ticker.values.tolist()
names = first_n_column.Nombre.values.tolist()

##LOGIN TWITTER

In [ ]:
#Create the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
#Set the access token an the access token secret
authenticate.set_access_token(accessToken, accessTokenSecret)
#Create the API object
api = tweepy.API(authenticate, wait_on_rate_limit=True)

##SCRAP FROM TWITTER

This function is in charge of extracting 1000 tweets from the term that is passed to it, in this case it will be the name of the company.

In [ ]:
def scrap_tweets(search_term):
  #Cursor object
  tweets = tweepy.Cursor(api.search, q=search_term, lang='en', since='2022-10-21', tweet_mode= 'extended').items(1000)
  #Store the tweets in a variable and get the full text
  all_tweets = [tweet.full_text for tweet in tweets]
  df = pd.DataFrame(all_tweets, columns=['Tweets'])  
  return df

##CREATE DATA FRAMES

Given a list, it will go through it and ask the previous function for the 1000 tweets that it will introduce in a dataframe called df_tweets_{stockName}.

In [ ]:
for i, name in enumerate(names):
  Dynamic_Variable_dfnum = f'df_tweets_{i}'
  globals()[Dynamic_Variable_dfnum] = scrap_tweets(names[i])

##CLEANING DATA

The following two functions will be in charge of cleaning the text of the dataframes with the tweets in order to leave in them only information of value and analyzable by the NLTK model.

In [ ]:
def cleanData(twt, st):
  twt = twt.to_string()
  twt = twt.lower()
  twt = re.sub(r'==.*?==+', ' ', twt)
  twt = twt.replace('\n', ' ')
  twt = twt.replace('\\n', ' ')
    twt = re.sub('@[A-Za-z0-9]+', ' ', twt) #Remove any string with a Hashtag
  
  return twt

In [ ]:
def token_stop_stemmed(text):
  
  tokenized_word = word_tokenize(text)
  tokenized_word = [w for w in tokenized_word if w.isalpha()]
  
  stop_words = set(stopwords.words("english"))
  
  filtered_word = [w for w in tokenized_word if w not in stop_words]
  
  ps = PorterStemmer()
  stemmed_words=[]
  for w in filtered_word:
    stemmed_words.append(ps.stem(w))

  return stemmed_words

The loop will loop through each of the dataframes created earlier with the Twitter search.

After passing through the first function it will return a string formed by the 1000 tweets clean of unnecessary symbols.

Once the string is passed to the second function it returns a list of words.

In [ ]:
for i, name in enumerate(names):
  
  dfnum_DV = f'df_tweets_{i}'
  text_DV = f'tweets_text_{i}'
  wordnum_DV = f'words_{i}'

  globals()[text_DV] = cleanData(globals()[dfnum_DV], name)

  globals()[wordnum_DV] = token_stop_stemmed(globals()[text_DV])

##SENTIMENT ANALYSIS

In [ ]:
def tweet_sentiment(text):
  sid = SentimentIntensityAnalyzer()
  stemmed_words_to_sentiment = ' '.join(text)
  sentiment = sid.polarity_scores(stemmed_words_to_sentiment)
  return sentiment

With a loop like the previous ones, we will pass the word lists of each of the 5 stocks to the function in charge of analyzing the sentiment of the tweets. 

In [ ]:
for i, name in enumerate(names):
  
  wordnum_DV = f'words_{i}'
  sentiment_DV = f'sentiment_{i}'

  globals()[sentiment_DV] = tweet_sentiment(globals()[wordnum_DV])

With the sentiment data that we have stored in a list for each of the stocks, we will extract the value corresponding to 'compound'.

In [ ]:
sentiment =[]

for i, name in enumerate(names):
  sentiment_DV = f'sentiment_{i}'

  sentiment.append(globals()[sentiment_DV] ['compound'])

##SENDING MESSAGE TO TELEGRAM

Finally, with the sentiment data and ticker for each stock, we compose a message to send to Telegram. 

La funcion decidira con el dato de 'compound' si el sentimiento es positivo, negativo o neutro. 

In [ ]:
def text_telegram(stock, sentiment):
  
  if sentiment > 0.3:
    text = f'👍  {stock} \n'
  elif sentiment < -0.3:
    text = f'❌ {stock} \n'
  else:
    text = f'↔ {stock} \n'
  
  return text

In [ ]:
bot = telebot.TeleBot(telegram_token)

In [ ]:
today = datetime.datetime.now() 
text = f'📈 Most volatile stock sentimens for {today.year}/{today.month}/{today.day} \n\n'

El bucle generará un texto para cada stock segun su sentimiento y fianlmente se enviará a Telegram.

In [ ]:
for i, stock in enumerate(tickers):

  text = text + text_telegram(stock, sentiment[i])

In [ ]:
bot.send_message(id_channel, text)